# Tutorial inicial 

A continuación desarrollaremos un tutorial introductorio para sistemas de recomendación disponible en: https://www.datacamp.com/community/tutorials/recommender-systems-python


In [39]:
import pandas as pd
import numpy as np 

## Recomendaciones simples

In [40]:
#cargamos los datos que usaremos
metadata = pd.read_csv('C:/Users/gpabl/Documents/git/proyectomat282/datos/movies_metadata.csv', low_memory=False)

In [25]:
metadata.head(2) #visualizamos la 2 primeras lineas del cuadro.

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


Se ocupa una metrica que mezcla el numero de votantes con la calificacion obtenida para asi asegurar que una pelicula con muchos votantes tenga un valor mayor que una pelicula con muy pocos votantes y ambas con igual calificación. una metrica que cumple con estos requisitos es $$ WeightedRating = WR = WR(v,R,m.C) = \left(\dfrac{v}{v+m}R\right) + \left(\dfrac{m}{v+m}C\right) $$ donde $v$ es el numero de votos de la pelicula, $m$ es el numero minimo de votos necesarios para ser incluida en el cuadro (a elección), $R$ es la calificacion promedio de la pelicula y $C$ es ¿el promedio de votos de todo el informe?.

los valores de $v,R$ estan directamente en la tabla, $C$ lo podemos calcular y $m$ es un valor a elegir,correspondiente por ejemplo a un percentil dado. en el tutorial usan percentil=0.9. De esta manera 

In [26]:
C = metadata['vote_average'].mean() #calculamos el valor de C
percentil = 0.9 # definimos el percentil deseado
m = metadata['vote_count'].quantile(percentil) #Calculamos el percentil antes definido.
display(C)
display(m)



5.618207215134185

160.0

In [27]:
# cramos un dataframe auxiliar con las peliculas que cumplen con el minimo de votos
q_movies = metadata.copy().loc[metadata['vote_count'] >= m]


El numero de peliculas en el dataframe filtrado es mucho menor que el original pues 

In [28]:
display(q_movies.shape)
display(metadata.shape)

(4555, 24)

(45466, 24)

Posteriormente hacemos una funcion que nos permita calcular la metrica antes vista

In [29]:
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

y posteriormente agregamos una columna llamada "score" al data frame auxiliar q_movies  utilizando la funcion anterior

In [34]:
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

In [35]:
q_movies.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,score
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,7.640253
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,6.820293


Y lo ordenamos de forma descendiente respecto al score.

In [36]:
q_movies = q_movies.sort_values('score', ascending=False)

In [38]:
#filtramos convenientemente y visualizamos las 15 mejores peliculas
q_movies[['title', 'vote_count', 'vote_average', 'score']].head(15)

,title,vote_count,vote_average,score
314,The Shawshank Redemption,8358.0,8.5,8.445869
834,The Godfather,6024.0,8.5,8.425439
10309,Dilwale Dulhania Le Jayenge,661.0,9.1,8.421453
12481,The Dark Knight,12269.0,8.3,8.265477
2843,Fight Club,9678.0,8.3,8.256385
292,Pulp Fiction,8670.0,8.3,8.251406
522,Schindler's List,4436.0,8.3,8.206639
23673,Whiplash,4376.0,8.3,8.205404
5481,Spirited Away,3968.0,8.3,8.196055
2211,Life Is Beautiful,3643.0,8.3,8.187171


## Content-Based Recommender